In [ ]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import time
import os
import sys
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import shutil
import tempfile
import base64
import io
import zipfile
from IPython.display import FileLink
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Set the default renderer to 'notebook' for Jupyter
pio.renderers.default = 'notebook'

from IPython.display import HTML, display
display(HTML("<script>document.title='NOMAD JV Analyzer'</script>"))

# These imports are from your existing code - adjust paths as needed
from main import load_files, find_unique_values, data_filter_setup, plotting_string_action, save_full_data_frame
sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_sample_description, get_all_JV, get_all_measurements_except_JV
from batch_selection import create_batch_selection

# --- Global Application State ---
current_token = None
current_user_info = None
data = {}  # Store for loaded data
is_conditions = False
unique_vals = []
global_plot_data = {'figs': [], 'names': [], 'workbook': None}
warning_sign = "⚠️"
batch_selection_container = widgets.Output()

# --- URL Configuration (Fixed to SE Oasis) ---
url_base = "http://localhost" # ⚠️⚠️⚠️⚠️⚠️⚠️ enter deployment url here ⚠️⚠️⚠️⚠️⚠️⚠️
url = f"{url_base}/nomad-oasis/api/v1"

# --- Helper Function for API Calls ---
def make_api_request(method, url, headers=None, params=None, json_data=None, timeout=10):
    """Makes an API request and handles common errors."""
    if not current_token:
        raise ValueError("Authentication token is missing.")

    request_headers = headers.copy() if headers else {}
    request_headers['Authorization'] = f'Bearer {current_token}'

    try:
        response = requests.request(method, url, headers=request_headers, params=params, json=json_data, timeout=timeout)
        response.raise_for_status()
        if response.text:
            return response.json()
        return None
    except requests.exceptions.RequestException as e:
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get('detail', e.response.text)
                if isinstance(error_detail, list):
                    error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                else:
                    error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                error_message = f"API Error ({e.response.status_code}): {e.response.text}"
        raise ConnectionError(error_message) from e
    except Exception as e:
        raise Exception(f"Unexpected Error during API request: {e}") from e

# --- Authentication Widgets ---
auth_method_selector = widgets.RadioButtons(
    options=['Username/Password', 'Token (from ENV)'],
    description='Auth Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px 0 0 0')
)

username_input = widgets.Text(
    placeholder='Enter Username (e.g., email)',
    description='Username:',
    style={'description_width': 'initial'}
)

password_input = widgets.Password(
    placeholder='Enter Password',
    description='Password:',
    style={'description_width': 'initial'}
)

token_input = widgets.Password(
    placeholder='Token will be read from ENV',
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%'),
    disabled=True
)

local_auth_box = widgets.VBox([username_input, password_input], layout=widgets.Layout(margin='5px 0 0 0'))
token_auth_box = widgets.VBox([token_input], layout=widgets.Layout(margin='5px 0 0 0', display='none'))

auth_button = widgets.Button(
    description='Authenticate',
    button_style='info',
    tooltip='Authenticate using the selected method',
    icon='unlock',
    layout=widgets.Layout(width='auto', margin='10px 0 0 0')
)

auth_status_label = widgets.Label(
    value='Status: Not Authenticated',
    layout=widgets.Layout(margin='5px 0 0 0')
)

auth_action_box = widgets.VBox([auth_button, auth_status_label])

# --- Authentication Method Change Observer ---
def on_auth_method_change(change):
    global current_token, current_user_info
    if change['type'] == 'change' and change['name'] == 'value':
        selected_method = change['new']
        if selected_method == 'Username/Password':
            local_auth_box.layout.display = 'flex'
            token_auth_box.layout.display = 'none'
        else:  # Token method
            local_auth_box.layout.display = 'none'
            token_auth_box.layout.display = 'none'
        
        auth_status_label.value = 'Status: Not Authenticated (Method changed)'
        auth_status_label.style.text_color = None
        current_token = None
        current_user_info = None

auth_method_selector.observe(on_auth_method_change, names='value')

# --- Authentication Button Handler ---
def on_auth_button_clicked(b):
    global current_token, current_user_info
    current_token = None
    current_user_info = None
    auth_status_label.value = 'Status: Authenticating...'
    auth_status_label.style.text_color = 'orange'

    try:
        if auth_method_selector.value == 'Username/Password':
            username = username_input.value
            password = password_input.value
            if not username or not password:
                raise ValueError("Username and Password are required.")

            auth_dict = dict(username=username, password=password)
            token_url = f"{url}/auth/token"
            response = requests.get(token_url, params=auth_dict, timeout=10)
            response.raise_for_status()

            token_data = response.json()
            if 'access_token' not in token_data:
                raise ValueError("Access token not found in response.")

            current_token = token_data['access_token']
            password_input.value = ''

        elif auth_method_selector.value == 'Token (from ENV)':
            token_from_env = os.environ.get('NOMAD_CLIENT_ACCESS_TOKEN')
            if not token_from_env:
                raise ValueError("Token not found in environment variable 'NOMAD_CLIENT_ACCESS_TOKEN'.")
            current_token = token_from_env
            
        # Verify Token
        if current_token:
            verify_url = f"{url}/users/me"
            headers = {'Authorization': f'Bearer {current_token}'}
            verify_response = requests.get(verify_url, headers=headers, timeout=10)
            verify_response.raise_for_status()

            current_user_info = verify_response.json()
            user_display = current_user_info.get('name', current_user_info.get('username', 'Unknown User'))
            auth_status_label.value = f'Status: Authenticated as {user_display} on SE Oasis.'
            auth_status_label.style.text_color = 'green'
            
            # Enable the first tab if authentication is successful
            tabs.selected_index = 0  # Switch to the first tab after successful auth
            enable_tab(0)  # Enable the first tab
            
            # Close the settings box after successful authentication
            close_settings_box()
            
            # Initialize batch selection AFTER successful authentication
            init_batch_selection()

    except ValueError as ve:
        auth_status_label.value = f'Status: Error - {ve}'
        auth_status_label.style.text_color = 'red'
        current_token = None
        current_user_info = None
    except requests.exceptions.RequestException as e:
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get('detail', e.response.text)
                if isinstance(error_detail, list):
                    error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                else:
                    error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                error_message = f"API Error ({e.response.status_code}): {e.response.text}"

        auth_status_label.value = f'Status: {error_message}'
        auth_status_label.style.text_color = 'red'
        current_token = None
        current_user_info = None
    except Exception as e:
        auth_status_label.value = f'Status: Unexpected Error - {e}'
        auth_status_label.style.text_color = 'red'
        current_token = None
        current_user_info = None

# Function to run authentication automatically on startup
def auto_authenticate():
    """Automatically run authentication when the notebook starts"""
    # print("Starting automatic authentication...")
    on_auth_button_clicked(None)

auth_button.on_click(on_auth_button_clicked)

# --- Combine Authentication Widgets ---
auth_box = widgets.VBox([
    auth_method_selector,
    local_auth_box,
    token_auth_box,
    auth_action_box
])

# --- Create collapsible settings box ---
settings_toggle_button = widgets.Button(
    description='▼ Connection Settings',
    button_style='',
    layout=widgets.Layout(width='200px', margin='0 0 5px 0'),
    style={'font_weight': 'bold'}
)

settings_content = widgets.VBox([
    widgets.HTML("<p><strong>SE Oasis:</strong> https://nomad-hzb-se.de/nomad-oasis/api/v1</p>"),
    auth_box
], layout=widgets.Layout(padding='10px', margin='0 0 10px 0'))

# Initially visible settings content
general_settings_box = widgets.VBox([
    settings_toggle_button,
    settings_content
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='0 0 20px 0'))

# Function to toggle settings visibility
def toggle_settings(b):
    if settings_content.layout.display == 'none':
        # Show settings
        settings_content.layout.display = 'flex'
        settings_toggle_button.description = '▼ Connection Settings'
    else:
        # Hide settings
        settings_content.layout.display = 'none'
        settings_toggle_button.description = '▶ Connection Settings'

settings_toggle_button.on_click(toggle_settings)

# Function to close settings box (called after successful auth)
def close_settings_box():
    settings_content.layout.display = 'none'
    settings_toggle_button.description = '▶ Connection Settings'

# --- Initial Environment Detection ---
is_hub_environment = bool(os.environ.get('JUPYTERHUB_USER'))

if is_hub_environment:
    auth_method_selector.value = 'Token (from ENV)'
    local_auth_box.layout.display = 'none'
    token_auth_box.layout.display = 'none'
else:
    auth_method_selector.value = 'Username/Password'
    local_auth_box.layout.display = 'flex'
    token_auth_box.layout.display = 'none'

# --- Tab 1: Select Upload ---
# Re-using your existing widgets with adjusted layouts
load_status_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# Tab 1 Handler Functions
def init_batch_selection():
    with batch_selection_container:
        clear_output(wait=True)
        if not current_token:
            print("Authentication required. Please authenticate first in Connection Settings.")
            return
        
        try:
            # Debug output
            print(f"Initializing batch selection with token: {current_token[:5]}...")
            
            # Create the batch selection widget using the imported function
            batch_selection_widget = create_batch_selection(url, current_token, load_data_from_selection)
            display(batch_selection_widget)
        except Exception as e:
            print(f"Error initializing batch selection: {e}")
            import traceback
            traceback.print_exc()

def load_data_from_selection(batch_selector):
    global data, unique_vals
    data = {}
    
    with load_status_output:
        clear_output(wait=True)
        print("Loading Data")
        
        if not current_token:
            print("Authentication required. Please authenticate first in Connection Settings.")
            return
            
        if not batch_selector.value:
            print("Please select at least one batch to load.")
            return
        
        try:
            batch_ids_list = batch_selector.value  # This should be a list of batch IDs
            print(f"Loading data for batch IDs: {batch_ids_list}")
            
            sample_ids = get_ids_in_batch(url, token=current_token, batch_ids=batch_ids_list)
            identifiers = get_sample_description(url, token=current_token, sample_ids=sample_ids)              
            df_jvc, df_cur = get_jv_data_for_analysis(sample_ids)
            data["jvc"] = pd.concat([data.get("jvc", pd.DataFrame()), df_jvc], ignore_index=True)
            data["curves"] = pd.concat([data.get("curves", pd.DataFrame()), df_cur], ignore_index=True)
            
            data["jvc"]["subbatch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-2])
            data["jvc"]["batch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-3])
            data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: '_'.join(x.split('/')[-1].split('_')[3:-1]))
            data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split(".")[0])
            
            if identifiers:
                data["jvc"]["identifier"] = data["jvc"]["identifier"].apply(lambda x: f'{"_".join(x.split("_")[:-1])}&{identifiers.get(x,"No variation specified")}')
            else:
                data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: "_".join(x.split('/')[-1].split(".")[0].split("_")[:-1]))
                  
            data["jvc"]["sample"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
            data["curves"]["sample"] = data["curves"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
            
            print("Data Loaded Successfully!")
            
            # Export data to CSV files
            df_jvc.to_csv("export_jvc.csv")
            df_cur.to_csv("export_cur.csv")
            
            # Find unique values for variables
            unique_vals = find_unique_values(data["jvc"])
            
            # Enable and switch to the next tab
            enable_tab(1)
            tabs.selected_index = 1
            
            # Initialize variables tab with the loaded data
            make_variables_menu(unique_vals)
            
        except Exception as e:
            print(f"Error loading data: {e}")
            import traceback
            traceback.print_exc()
            
def on_load_data_clicked(b):
    global data, unique_vals
    data = {}
    
    with load_status_output:
        clear_output(wait=True)
        print("Loading Data")
        
        if not current_token:
            print("Authentication required. Please authenticate first in Connection Settings.")
            return
            
        if not batch_ids.value:
            print("Please select at least one batch to load.")
            return
        
        try:
            sample_ids = get_ids_in_batch(url, token=current_token, batch_ids=batch_ids.value)
            identifiers = get_sample_description(url, token=current_token, sample_ids=sample_ids)              
            df_jvc, df_cur = get_jv_data_for_analysis(sample_ids)
            data["jvc"] = pd.concat([data.get("jvc", pd.DataFrame()), df_jvc], ignore_index=True)
            data["curves"] = pd.concat([data.get("curves", pd.DataFrame()), df_cur], ignore_index=True)
            
            data["jvc"]["subbatch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-2])
            data["jvc"]["batch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-3])
            data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: '_'.join(x.split('/')[-1].split('_')[3:-1]))
            data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split(".")[0])
            
            if identifiers:
                data["jvc"]["identifier"] = data["jvc"]["identifier"].apply(lambda x: f'{"_".join(x.split("_")[:-1])}&{identifiers.get(x,"No variation specified")}')
            else:
                data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: "_".join(x.split('/')[-1].split(".")[0].split("_")[:-1]))
                  
            data["jvc"]["sample"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
            data["curves"]["sample"] = data["curves"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
            
            print("Data Loaded Successfully!")
            
            # Export data to CSV files
            df_jvc.to_csv("export_jvc.csv")
            df_cur.to_csv("export_cur.csv")
            
            # Find unique values for variables
            unique_vals = find_unique_values(data["jvc"])
            
            # Enable and switch to the next tab
            enable_tab(1)
            tabs.selected_index = 1
            
            # Initialize variables tab with the loaded data
            make_variables_menu(unique_vals)
            
        except Exception as e:
            print(f"Error loading data: {e}")

#load_data_button.on_click(on_load_data_clicked)

def get_jv_data_for_analysis(sample_ids):
    columns_jvc = ['Voc(V)','Jsc(mA/cm2)','FF(%)','PCE(%)','V_mpp(V)','J_mpp(mA/cm2)',
                  'P_mpp(mW/cm2)','R_series(Ohmcm2)','R_shunt(Ohmcm2)','sample','batch',
                  'condition','cell','direction','ilum']
    columns_cur = ['index','sample','batch','condition','variable','cell','direction','ilum']
    rows_jvc = []
    rows_cur = []
    
    with load_status_output:
        all_jvs = get_all_JV(url, current_token, sample_ids)
        for sid in sample_ids:
            jv_res = all_jvs.get(sid,[])
            clear_output(wait=True)
            print("Processing: ", sid)
            for jv_data, jv_md in jv_res:
                for c in jv_data["jv_curve"]:
                    file_name = os.path.join("../", jv_md["upload_id"], jv_data.get("data_file"))
                    illum = "Dark" if "dark" in c["cell_name"].lower() else "Light"
                    cell = c["cell_name"][0]
                    direction = "Forward" if "for" in c["cell_name"].lower() else "Reverse"
                    row = [c["open_circuit_voltage"],-c["short_circuit_current_density"],100*c["fill_factor"],c["efficiency"],c["potential_at_maximum_power_point"],-c["current_density_at_maximun_power_point"],
                           -c["potential_at_maximum_power_point"]*c["current_density_at_maximun_power_point"],c["series_resistance"],c["shunt_resistance"],file_name,file_name.split("/")[1],"w",
                           cell,direction, illum]
                    rows_jvc.append(row)
                    row_v = ["_".join(["Voltage (V)",cell,direction,illum]),file_name,file_name.split("/")[1],"w","Voltage (V)", 
                             cell,direction, illum]
                    row_v.extend(c["voltage"])
                    row_j = ["_".join(["Current Density(mA/cm2)",cell,direction,illum]),file_name,file_name.split("/")[1],"w","Current Density(mA/cm2)", 
                             cell,direction, illum]
                    row_j.extend(c["current_density"])
                    for i in range(len(row_v[8:])):
                        if i not in columns_cur:
                            columns_cur.append(i)

                    rows_cur.append(row_v)
                    rows_cur.append(row_j)
    
    df_jvc = pd.DataFrame(rows_jvc, columns=columns_jvc)
    df_cur = pd.DataFrame(rows_cur, columns=columns_cur)
    return df_jvc, df_cur

# Organize Tab 1 Layout
select_upload_tab = widgets.VBox([
    widgets.HTML("<h3>Select Upload</h3>"),
    batch_selection_container,
    load_status_output
])

# --- Tab 2: Add Variable Names ---
default_variables = widgets.Dropdown(
    options=['all', 'Batch name', 'Variation'],
    value='Variation',
    description='Defaults:',
    disabled=False,
    style={'description_width': 'initial'}
)

variables_output = widgets.Output(layout=widgets.Layout(min_height='200px'))
dynamic_content = widgets.Output()
results_content = widgets.Output(layout={
    'width': '400px',
    'height': '300px',
    'overflow': 'scroll',
})
read = widgets.Output()

# Tab 2 Handler Functions
def create_widgets_table(elements_list):
    rows = []
    text_widgets = {}
    for item in elements_list:
        item_split = item.split("&")
        batch, variable = "", item
        if len(item_split) >= 2:
            batch, variable = item_split[0], "&".join(item_split[1:])
        default_value = ""
        if default_variables.value == "Batch name":
            default_value = batch if batch else "_".join(item.split("_")[:-1])
        if default_variables.value == "Variation":
            default_value = variable
        label = widgets.Label(value=variable)
        text_input = widgets.Text(value=default_value, placeholder='Variable e.g. 1000 rpm')
        row = widgets.HBox([label, text_input])
        rows.append(row)
        text_widgets[item] = text_input  
    return widgets.VBox(rows), text_widgets

def on_retrieve_clicked(text_widgets_dict):
    global is_conditions
    is_conditions = True
    conditions_dict = {}
    read.clear_output()
    for item, text_widget in text_widgets_dict.items():
        conditions_dict[item] = text_widget.value
    data['jvc']['condition'] = data['jvc']['identifier'].map(conditions_dict)
    
    with read:
        print("Variables loaded")
    
    # Show download button and measurements
    # FIX: Use the download_content Output widget instead of directly using download_area
    with download_content:
        clear_output(wait=True)
        display(download_button)
    
    # Automatically show measurements
    show_table(None)
    
    # Enable the next tab
    enable_tab(2)
    tabs.selected_index = 2

def brief_data_summary(df):
    global_mean_PCE = df['PCE(%)'].mean()
    global_std_PCE = df['PCE(%)'].std()
    max_PCE_row = df.loc[df['PCE(%)'].idxmax()]
    mean_std_PCE_per_sample = df.groupby(['batch', 'sample'])['PCE(%)'].agg(['mean', 'std'])
    highest_mean_PCE_sample = mean_std_PCE_per_sample.idxmax()['mean']
    lowest_mean_PCE_sample = mean_std_PCE_per_sample.idxmin()['mean']
    highest_PCE_per_sample = df.loc[df.groupby(['sample'])['PCE(%)'].idxmax(), ['sample', 'cell', 'PCE(%)']]
    
    markdown_output = f"""
### Summary Statistics

**Global mean PCE(%)**: {global_mean_PCE:.2f} ± {global_std_PCE:.2f}%


|   | Sample | Mean PCE(%) | Std PCE(%) |
|---|--------|-------------|------------|
| Best sample | {highest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'std']:.2f}% |
| Worst sample | {lowest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'std']:.2f}% |

#### Highest PCE(%) per Sample

| | Sample | Cell | PCE(%) |
|-|--------|------|--------|
| Best Overall | {max_PCE_row['sample']} | {max_PCE_row['cell']} | {max_PCE_row['PCE(%)']:.2f}% |
"""
    for sample, row in highest_PCE_per_sample.set_index('sample').iterrows():
        markdown_output += f"| | {sample} | {row['cell']} | {row['PCE(%)']:.2f}% |\n"
        
    return markdown_output

def make_variables_menu(unique_vals):
    variables_markdown = f"""
# Add variable names
There are {len(unique_vals)} samples found.
If you tested specific variables or conditions for each sample, please write them down below.
""" 
    results_markdown = brief_data_summary(data['jvc'])
    with dynamic_content:
        clear_output(wait=True)
        display(Markdown(variables_markdown))
        display(default_variables)
        widgets_table, text_widgets_dict = create_widgets_table(unique_vals)
        retrieve_button = widgets.Button(
            description="Confirm variables",
            button_style='success',
            layout=widgets.Layout(min_width='150px')
        )
        retrieve_button.on_click(lambda b: on_retrieve_clicked(text_widgets_dict))
        information_group = widgets.HBox([widgets_table, results_content])
        display(information_group)
        button_group = widgets.HBox([retrieve_button, read])
        display(button_group)
    
    results_html = widgets.HTML(value=f"<div>{results_markdown}</div>")
    with results_content:
        results_content.clear_output()
        display(Markdown(results_markdown))
    with read:
        read.clear_output()
        print(f"{warning_sign} Variables not loaded")
        
def on_change_default_variables(b):
    dynamic_content.clear_output()
    global unique_vals
    make_variables_menu(unique_vals)

default_variables.observe(on_change_default_variables, names=['value'])

# Quick download section
download_button = widgets.Button(
    description='Download JV',
    button_style='info',
    layout=widgets.Layout(min_width='150px')
)

download_content = widgets.Output()
download_area = widgets.VBox([download_content])

def trigger_download(text, filename, kind='text/json'):
    content_b64 = base64.b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_content:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

def download_jv_data(e=None):
    jvc = io.StringIO()
    curves = io.StringIO()
    data["jvc"].to_csv(jvc)
    data["curves"].to_csv(curves)
    trigger_download(jvc.getvalue(), 'jvc.csv', kind='text/plain')
    trigger_download(curves.getvalue(), 'curves.csv', kind='text/plain')

download_button.on_click(download_jv_data)

# Show measurements section
show_other_measurements = widgets.Output()
measurements_area = widgets.VBox([show_other_measurements])

def show_table(e=None):
    if not current_token:
        with show_other_measurements:
            clear_output(wait=True)
            print("Please authenticate and load data first.")
        return
        
    with show_other_measurements:
        clear_output(wait=True)
        print("Loading measurements data...")
        
        try:
            # Get the selected batch IDs from the loaded data
            batch_ids_value = list(data['jvc']['batch'].unique()) if 'jvc' in data and 'batch' in data['jvc'] else []
            
            if not batch_ids_value:
                print("No batch IDs found in loaded data.")
                return
                
            sample_ids = get_ids_in_batch(url, current_token, batch_ids_value)
            measurements_data = get_all_measurements_except_JV(url, current_token, sample_ids)

            df = pd.DataFrame()
            def make_clickable(r):
                if "SEM" in r[1]["entry_type"]:
                    return f'<a href="{url_base}/nomad-oasis/gui/entry/id/{r[1]["entry_id"]}/data/data/images:0/image_preview/preview" rel="noopener noreferrer" target="_blank" >{r[1]["entry_type"].split("_")[-1]}</a>'
                return f'<a href="{url_base}/nomad-oasis/gui/entry/id/{r[1]["entry_id"]}/data/data" rel="noopener noreferrer" target="_blank" >{r[1]["entry_type"].split("_")[-1]}</a>'

            for key, value in measurements_data.items():
                df[key] = pd.Series([make_clickable(r) for r in value])
            
            if df.empty:
                print("No additional measurements found.")
            else:
                display(HTML("<h3>Additional Measurements</h3>"))
                display(HTML(df.to_html(escape=False)))
        
        except Exception as e:
            print(f"Error displaying measurements: {e}")

# Organize Tab 2 Layout
add_variables_tab = widgets.VBox([
    widgets.HTML("<h3>Add Variable Names</h3>"),
    dynamic_content,
    widgets.HTML("<h3>Download Data</h3>"),
    download_area,
    widgets.HTML("<h3>Other Measurements</h3>"),
    measurements_area
])

# --- Tab 3: Select Filters ---
filter_presets = {
    "Default": [(["PCE(%)", "<", "40"]), (["FF(%)", "<", "89"]), (["FF(%)", ">", "24"]), (["Voc(V)", "<", "2"]), (["Jsc(mA/cm2)", ">", "-30"])],
    "Preset 2": [(["FF(%)", "<", "15"]), (["PCE(%)", ">=", "10"])]
}

# Create filter widgets
preset_dropdown_filter = widgets.Dropdown(
    options=list(filter_presets.keys()),
    description='Filters',
    layout=widgets.Layout(width='200px')
)

direction_radio = widgets.RadioButtons(
    options=['Both', 'Reverse', 'Forward'],
    value='Both',
    description='Direction:',
    layout=widgets.Layout(width='200px')
)

add_button_filter = widgets.Button(description="Add Filter", button_style='primary')
remove_button_filter = widgets.Button(description="Remove Filter", button_style='danger')
apply_preset_filter_button = widgets.Button(description="Load Preset", button_style='info')
apply_filter_button = widgets.Button(
    description="Apply Filter", 
    button_style='success',
    layout=widgets.Layout(min_width='150px')
)
confirmation_filter = widgets.Output()
main_output_content = widgets.Output(layout={
    'width': '400px',
    'height': '250px',
    'overflow': 'scroll',
})

def create_widget_group_filter(include_headers=False):
    headers = widgets.HBox([
        widgets.Label('Variable', layout=widgets.Layout(width='80px')),
        widgets.Label('Operator', layout=widgets.Layout(width='80px')),
        widgets.Label('Value', layout=widgets.Layout(width='auto'))
    ]) if include_headers else None

    dropdown1 = widgets.Dropdown(
        options=['Voc(V)', 'Jsc(mA/cm2)', 'FF(%)', 'PCE(%)', 'V_MPP(V)', 'J_MPP(mA/cm2)'],
        description='', 
        layout=widgets.Layout(width='80px')
    )
    dropdown2 = widgets.Dropdown(
        options=['>', '>=', '<', '<=', '==', '!='], 
        description='', 
        layout=widgets.Layout(width='80px')
    )
    text_input = widgets.Text(
        value='', 
        placeholder='Write a value', 
        description='', 
        layout=widgets.Layout(width='100px')
    )
    row = widgets.HBox([dropdown1, dropdown2, text_input])
    return widgets.VBox([headers, row]) if include_headers else row

widget_groups_filter = [create_widget_group_filter(include_headers=False)]
groups_container_filter = widgets.VBox(widget_groups_filter)

# Tab 3 Handler Functions
def add_widget_group_filter(b):
    widget_groups_filter.append(create_widget_group_filter())
    groups_container_filter.children = widget_groups_filter

def remove_widget_group_filter(b):
    if len(widget_groups_filter) > 1:
        widget_groups_filter.pop()
        groups_container_filter.children = widget_groups_filter

def apply_preset_filter(b):
    selected_preset = preset_dropdown_filter.value
    widget_groups_filter.clear()
    if selected_preset in filter_presets:
        for variable, operator, value in filter_presets[selected_preset]:
            group = create_widget_group_filter()
            group.children[0].value = variable
            group.children[1].value = operator
            group.children[2].value = value
            widget_groups_filter.append(group)
    else:
        widget_groups_filter.append(create_widget_group_filter(include_headers=False))
    groups_container_filter.children = widget_groups_filter

def apply_filtering(b):
    global filter_vals, data
    filter_values = []
    main_output_content.clear_output()
    
    for group in widget_groups_filter:
        variable = group.children[0].value
        operator = group.children[1].value
        value = group.children[2].value
        filter_values.append((variable, operator, value))
    
    direction_value = direction_radio.value
    confirmation_filter.clear_output()
    
    with main_output_content:
        try:
            # Apply the regular numeric filters
            data['filtered'], data['junk'], filter_vals = data_filter_setup(data['jvc'], filter_values)
            
            # Apply direction filter separately if not "Both"
            if direction_value != 'Both':
                data['filtered'] = data['filtered'][data['filtered']['direction'] == direction_value]
                direction_mask = data['jvc']['direction'] != direction_value
                data['junk'] = pd.concat([
                    data['junk'], 
                    data['jvc'][direction_mask & ~data['jvc'].index.isin(data['junk'].index)]
                ])
            
            print(f"Filtering complete! Retained {len(data['filtered'])} records, filtered out {len(data['junk'])} records.")
            print("You can proceed to the next tab to create plots.")
            
            # Enable the next tab
            enable_tab(3)
            #tabs.selected_index = 3
            
        except Exception as e:
            print(f"Error applying filters: {e}")

# Connect handlers to buttons
add_button_filter.on_click(add_widget_group_filter)
remove_button_filter.on_click(remove_widget_group_filter)
apply_preset_filter_button.on_click(apply_preset_filter)
apply_filter_button.on_click(apply_filtering)

# Initialize with preset
apply_preset_filter(None)

# Filter tab layout components
direction_label = widgets.HTML(value="<b>Filter by Cell Direction:</b>")
direction_container = widgets.VBox([direction_label, direction_radio])

filter_conditions_label = widgets.HTML(value="<b>Filter Conditions:</b>")
filter_conditions_container = widgets.VBox([filter_conditions_label, groups_container_filter])

confirm_group_filter = widgets.HBox([apply_filter_button, confirmation_filter])
controls_filter = widgets.VBox([
    add_button_filter, 
    remove_button_filter, 
    preset_dropdown_filter, 
    apply_preset_filter_button,
    confirm_group_filter
])

# Organize Tab 3 Layout
filter_tab_layout = widgets.GridspecLayout(1, 3)
filter_tab_layout[0, 0] = controls_filter
filter_tab_layout[0, 1] = widgets.VBox([direction_container, filter_conditions_container])
filter_tab_layout[0, 2] = main_output_content

select_filters_tab = widgets.VBox([
    widgets.HTML("<h3>Select Filters</h3>"),
    widgets.HTML("<p>Using the dropdowns below, select filters for the data you want to keep, not remove.</p>"),
    filter_tab_layout
])

# --- Tab 4: Select Plots ---
plot_presets = {
    "Default": [("Boxplot", "PCE", "by Variable"), ("Boxplot", "Voc", "by Variable"), ("Boxplot", "Jsc", "by Variable"),
                ("Boxplot", "FF", "by Variable"), ("JV Curve", "Best device only", "")],
    "Preset 2": [("Boxplot", "Voc", "by Cell"), ("Histogram", "Voc", ""), ("JV Curve", "Best device only", "")]
}

# Create plot widgets
load_preset_plot_button = widgets.Button(
    description="Load Preset",
    button_style='info'
)

preset_dropdown_plot = widgets.Dropdown(
    options=list(plot_presets.keys()),
    description='Presets'
)

add_button_plot = widgets.Button(
    description="Add Plot Type",
    button_style='primary'
)

remove_button_plot = widgets.Button(
    description="Remove Plot Type",
    button_style='danger'
)

plot_selection_button = widgets.Button(
    description="Plot Selection",
    button_style='success',
    layout=widgets.Layout(min_width='150px')
)

plotted_content = widgets.Output()

# Function to update additional options based on plot type selection
def update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown):
    plot_type = plot_type_dropdown.value
    if plot_type == 'Boxplot':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction']
    elif plot_type == 'Boxplot (omitted)':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction']
    elif plot_type == 'Histogram':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['']  # Histogram does not use the second option
    elif plot_type == 'JV Curve':
        option1_dropdown.options = ['All cells', 'Only working cells', 'Only not working cells', 'Best device only', 'Separated by cell', 'Separated by substrate']
        option2_dropdown.options = ['']  # JV Curve does not use the second option
    else:
        option1_dropdown.options = []
        option2_dropdown.options = []

# Function to create a new plot type group
def create_plot_type_group():
    plot_type_dropdown = widgets.Dropdown(
        options=['Boxplot', 'Boxplot (omitted)', 'Histogram', 'JV Curve'], 
        description='Plot Type:', 
        layout=widgets.Layout(width='250px')
    )
    option1_dropdown = widgets.Dropdown(
        description='Option 1:',
        layout=widgets.Layout(width='300px')
    )
    option2_dropdown = widgets.Dropdown(
        description='Option 2:',
        layout=widgets.Layout(width='300px')
    )

    # Initial update for default selection
    update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown)

    # Observer for plot type changes
    plot_type_dropdown.observe(
        lambda change: update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown), 
        names='value'
    )

    return widgets.HBox([plot_type_dropdown, option1_dropdown, option2_dropdown])

plot_type_groups = [create_plot_type_group()]
groups_container = widgets.VBox(plot_type_groups)

# Plot tab handler functions
def add_plot_type_group(b):
    new_group = create_plot_type_group()
    plot_type_groups.append(new_group)
    groups_container.children = tuple(plot_type_groups)

def remove_plot_type_group(b):
    if len(plot_type_groups) > 1:
        plot_type_groups.pop()
        groups_container.children = tuple(plot_type_groups)

def load_preset_plot(b):
    selected_preset = preset_dropdown_plot.value
    plot_type_groups.clear()
    if selected_preset in plot_presets:
        for plot_type, option1, option2 in plot_presets[selected_preset]:
            new_group = create_plot_type_group()
            new_group.children[0].value = plot_type  # Set plot type
            new_group.children[1].value = option1    # Set option 1
            new_group.children[2].value = option2    # Set option 2
            plot_type_groups.append(new_group)
    else:
        plot_type_groups.append(create_plot_type_group())
    groups_container.children = tuple(plot_type_groups)

def center_plot(fig):
    """Center the Plotly plot on the data"""
    # Get all visible traces
    all_x = []
    all_y = []
    
    for trace in fig.data:
        # Skip if trace has no data
        if trace.x is None or len(trace.x) == 0 or trace.y is None or len(trace.y) == 0:
            continue
            
        # Skip legend-only traces
        if trace.get('showlegend', True) and (trace.x[0] is None or trace.y[0] is None):
            continue
            
        # Add x and y data to lists
        all_x.extend([x for x in trace.x if x is not None])
        all_y.extend([y for y in trace.y if y is not None])
    
    # If we have data, center the plot
    if all_x and all_y:
        x_min, x_max = min(all_x), max(all_x)
        y_min, y_max = min(all_y), max(all_y)
        
        # Add a small margin (10%)
        x_margin = (x_max - x_min) * 0.1
        y_margin = (y_max - y_min) * 0.1
        
        # Update the axis ranges
        fig.update_xaxes(range=[x_min - x_margin, x_max + x_margin])
        fig.update_yaxes(range=[y_min - y_margin, y_max + y_margin])
        
    return fig

def add_center_button(fig):
    """Add a custom button to center the plot"""
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=[
                    dict(
                        args=[{
                            "xaxis.autorange": False, 
                            "yaxis.autorange": False,
                            # Center function logic
                            "xaxis.range": [
                                min([min(trace.x) for trace in fig.data if trace.x and len(trace.x) > 0]) * 0.9,
                                max([max(trace.x) for trace in fig.data if trace.x and len(trace.x) > 0]) * 1.1
                            ],
                            "yaxis.range": [
                                min([min(trace.y) for trace in fig.data if trace.y and len(trace.y) > 0]) * 0.9,
                                max([max(trace.y) for trace in fig.data if trace.y and len(trace.y) > 0]) * 1.1
                            ]
                        }],
                        label="Center Plot",
                        method="relayout"
                    )
                ],
                pad={"r": 10, "t": 10},
                showactive=False,
                x=0.98,
                xanchor="right",
                y=0.02,
                yanchor="bottom"
            )
        ]
    )
    return fig

def display_plotly_figure(fig, title=None):
    """Force display of a Plotly figure using direct HTML embedding"""
    try:
        from IPython.display import HTML, display
        
        # Convert the figure to HTML
        html_str = fig.to_html(include_plotlyjs='cdn', full_html=False)
        
        # Wrap in a div with a border for visibility
        wrapped_html = f"""
        <div style="border:1px solid #ccc; padding:10px; margin:10px 0;">
            <h3>{title if title else 'Figure'}</h3>
            {html_str}
        </div>
        """
        
        # Display using HTML
        display(HTML(wrapped_html))
        return True
    except Exception as e:
        print(f"Error displaying figure with HTML method: {e}")
        import traceback
        traceback.print_exc()
        return False

# Function to create a button that will center the plot on the data
def create_center_button(fig_list):
    """Create a button widget that centers all plots on their data"""
    button = widgets.Button(
        description='Center All Plots',
        tooltip='Click to center all plots on their data',
        button_style='info'
    )
    
    def on_button_clicked(b):
        for fig in fig_list:
            center_plot(fig)
    
    button.on_click(on_button_clicked)
    return button

# Store original home functions to avoid multiple overrides
original_home_functions = {}

def plot_selected_options(b):
    global global_plot_data

    extracted_contents = []
    for group in plot_type_groups:
        plot_type_widget, option1_widget, option2_widget = group.children[0], group.children[1], group.children[2]
        plot_type = plot_type_widget.value
        option1 = option1_widget.value
        option2 = option2_widget.value
        extracted_contents.append((plot_type, option1, option2))
    
    # Clear output with wait=True to prevent flickering
    with plotted_content:
        clear_output(wait=True)
        print("Creating plots, please wait...")
    
    try:
        if 'filtered' not in data:
            with plotted_content:
                clear_output(wait=True)
                print("Please apply filters first in the 'Select Filters' tab.")
            return
                
        voila = True
        wb = save_full_data_frame('', data['jvc'])

        data_plot = data['filtered'], data["jvc"], data["curves"]
        p_rel = os.getcwd()
        supp_plot = data['junk'], filter_vals, is_conditions, p_rel, data["curves"]["sample"].unique().tolist()

        # Get figures from plotting function
        figs, names, workbook = plotting_string_action(extracted_contents, wb, data_plot, supp_plot, voila)

        # Store the outputs in the global variable
        global_plot_data['figs'] = figs
        global_plot_data['names'] = names
        global_plot_data['workbook'] = workbook

        # Clear output again and display plots
        with plotted_content:
            clear_output(wait=True)
            
            # Method 1: Direct display with go.FigureWidget (Recommended for Voila)
            print(f"Displaying {len(figs)} plots...")
            
            for i, (fig, name) in enumerate(zip(figs, names)):
                try:
                    print(f"\nPlot {i+1}: {name}")
                    
                    # Convert to FigureWidget for better Voila compatibility
                    if hasattr(fig, 'data'):  # Check if it's a Plotly figure
                        # Create a new FigureWidget from the figure
                        fig_widget = go.FigureWidget(fig)
                        
                        # Update layout for better display in Voila
                        fig_widget.update_layout(
                            autosize=True,
                            margin=dict(l=50, r=50, t=50, b=50)
                        )
                        
                        # Display the widget
                        display(widgets.HTML(f"<h3>{name}</h3>"))
                        display(fig_widget)
                    else:
                        # Fallback for matplotlib figures
                        display(widgets.HTML(f"<h3>{name}</h3>"))
                        display(fig)
                        
                except Exception as e:
                    print(f"Error displaying plot {i+1} ({name}): {e}")
                    # Try alternative display method
                    try:
                        display(widgets.HTML(f"<h3>{name} (Alternative Display)</h3>"))
                        display(fig)
                    except:
                        print(f"Could not display plot {name}")
            
            print("\nPlots created successfully! Proceed to the next tab to save your results.")
            
            # Enable the save tab
            enable_tab(4)
            
    except Exception as e:
        with plotted_content:
            clear_output(wait=True)
            print(f"Error creating plots: {e}")
            import traceback
            traceback.print_exc()
            print("Please make sure you have applied filters in the previous tab.")

plot_selection_button.on_click(plot_selected_options)

# Connect handlers to buttons
add_button_plot.on_click(add_plot_type_group)
remove_button_plot.on_click(remove_plot_type_group)
load_preset_plot_button.on_click(load_preset_plot)
plot_selection_button.on_click(plot_selected_options)

# Initialize with default preset
load_preset_plot(None)

# Organize Tab 4 Layout
controls_plot = widgets.VBox([
    add_button_plot, 
    remove_button_plot, 
    preset_dropdown_plot,
    load_preset_plot_button,
    plot_selection_button
])

select_plots_tab = widgets.VBox([
    widgets.HTML("<h3>Select Plots</h3>"),
    widgets.HTML("<p>Using the dropdowns below, select the plots you want to create.</p>"),
    widgets.HBox([controls_plot, groups_container]),
    plotted_content
])

# --- Tab 5: Save Plots and Data ---
save_markdown = f"""
# Save plots and data
"""

save_plots_button = widgets.Button(
    description='Save All Plots',
    tooltip='Click to save all plots',
    button_style='primary'
)

save_data_button = widgets.Button(
    description='Save Data',
    tooltip='Click to save the collected raw data',
    button_style='info'
)

save_all_button = widgets.Button(
    description='Save Data & Plots',
    tooltip='Click to save the collected raw data and all plots',
    button_style='success'
)

download_save_output = widgets.Output()

# Handler functions for saving plots and data
def on_save_plots_button_clicked(b):
    download_save_output.clear_output()

    if not global_plot_data.get('figs'):
        with download_save_output:
            print("No plots have been created yet. Please create plots in the previous tab.")
        return

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for i, (fig, name) in enumerate(zip(global_plot_data['figs'], global_plot_data['names'])):
            try:
                # Save as HTML
                html_str = fig.to_html(include_plotlyjs='cdn')
                zip_file.writestr(f"{name}", html_str)
                
                # Try to save as PNG if kaleido is installed
                try:
                    import plotly.io as pio
                    img_bytes = pio.to_image(fig, format='png')
                    zip_file.writestr(f"{name.replace('.html', '.png')}", img_bytes)
                except Exception as img_err:
                    print(f"Note: Could not save {name} as PNG: {img_err}")
                    # Don't let this stop the process
            except Exception as e:
                with download_save_output:
                    print(f"Error saving {name}: {e}")

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'plots.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all plots</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

def on_save_data_button_clicked(b):
    download_save_output.clear_output()

    if not global_plot_data.get('workbook'):
        with download_save_output:
            print("No data has been processed yet. Please create plots in the previous tab.")
        return

    # Save Excel file to memory
    excel_buffer = io.BytesIO()
    global_plot_data['workbook'].save(excel_buffer)
    excel_buffer.seek(0)

    # Encode to base64
    b64 = base64.b64encode(excel_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}';
    link.download = 'collected_data.xlsx';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download data</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

def on_save_all_button_clicked(b):
    download_save_output.clear_output()

    if not global_plot_data.get('figs') or not global_plot_data.get('workbook'):
        with download_save_output:
            print("No plots or data have been created yet. Please create plots in the previous tab.")
        return

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for i, (fig, name) in enumerate(zip(global_plot_data['figs'], global_plot_data['names'])):
            try:
                # Save as HTML
                html_str = fig.to_html(include_plotlyjs='cdn')
                zip_file.writestr(f"{name}", html_str)
                
                # Try to save as PNG if kaleido is installed
                try:
                    import plotly.io as pio
                    img_bytes = pio.to_image(fig, format='png')
                    zip_file.writestr(f"{name.replace('.html', '.png')}", img_bytes)
                except Exception as img_err:
                    print(f"Note: Could not save {name} as PNG: {img_err}")
                    # Don't let this stop the process
            except Exception as e:
                with download_save_output:
                    print(f"Error saving {name}: {e}")

        # Add Excel file to zip
        excel_buffer = io.BytesIO()
        global_plot_data['workbook'].save(excel_buffer)
        excel_buffer.seek(0)
        zip_file.writestr("collected_data.xlsx", excel_buffer.getvalue())

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'results.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all files</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

# Attach the event handlers to the buttons
save_plots_button.on_click(on_save_plots_button_clicked)
save_data_button.on_click(on_save_data_button_clicked)
save_all_button.on_click(on_save_all_button_clicked)

# Organize Tab 5 Layout
save_plots_and_data_tab = widgets.VBox([
    widgets.HTML("<h3>Save Plots and Data</h3>"),
    widgets.HBox([save_plots_button, save_data_button, save_all_button]),
    download_save_output
])

# --- Tab System Controller ---
# Dynamic tab enabling/disabling
tab_labels = ['Select Upload', 'Add Variable Names', 'Select Filters', 'Select Plots', 'Save Results']

# Create the tab widget
tabs = widgets.Tab()
tabs.children = [
    select_upload_tab, 
    add_variables_tab,
    select_filters_tab,
    select_plots_tab,
    save_plots_and_data_tab
]

# Set tab titles
for i, title in enumerate(tab_labels):
    tabs.set_title(i, title)

# Function to control tab enabling/disabling
def enable_tab(tab_index):
    """Enables the specified tab and updates styles"""
    # This function will be called when a step is completed to enable the next tab
    # In a real implementation, we would modify the tab style to show it's enabled
    # For this implementation, we'll just print a message
    #print(f"Tab '{tab_labels[tab_index]}' is now enabled.")
    
    # In a real implementation with CSS:
    # tabs.children[tab_index].layout.border = '2px solid green'
    pass

# Initially, only the first tab should be enabled
for i in range(1, len(tab_labels)):
    # In a real implementation:
    # tabs.children[i].layout.border = '2px solid gray'
    pass

# --- Main Dashboard Layout ---
app_layout = widgets.Layout(
    max_width="1200px",
    margin="0 auto",
    padding='15px'
)

dashboard_layout = widgets.VBox([
    widgets.HTML("<h1>JV Analysis Dashboard</h1>"),
    general_settings_box,
    tabs
], layout=app_layout)

# Display the dashboard
display(dashboard_layout)
auto_authenticate()

# --- Start with authentication tab ---
# Initial state: only authentication is enabled
enable_tab(0)  # Enable the first tab (Upload)
